In [ ]:
import os
import io
import contextlib
from dotenv import load_dotenv

load_dotenv()

In [ ]:
GOOGLE_GEMINI_API_KEY = os.environ.get("GOOGLE_GEMINI_API_KEY")
assert GOOGLE_GEMINI_API_KEY is not None

In [ ]:
BRIGHT_DATA_SERP_API_KEY = os.environ.get("BRIGHT_DATA_SERP_API_KEY")
assert BRIGHT_DATA_SERP_API_KEY is not None

In [ ]:
from langchain_brightdata import BrightDataSERP

serp_tool = BrightDataSERP(
    bright_data_api_key=BRIGHT_DATA_SERP_API_KEY,
    search_engine = 'google',
    country = 'in',
    parse_results = True
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key = GOOGLE_GEMINI_API_KEY)

# Simple text invocation
result = model.invoke("What is your name?")
print(result.content)

In [ ]:
def get_weather(city:str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# get_weather.name

In [ ]:
from langchain.agents import create_agent


weather_agent  = create_agent(
    model = model,
    tools = [get_weather],
    system_prompt = "You are a helpful assistant"
)

result = weather_agent.invoke(
    {"messages" : [{"role" : "user", "content" : "What is the weather in mannheim?"}]}
)

In [ ]:
result["messages"][-1].content

In [ ]:
serp_agent  = create_agent(
    model = model,
    tools = [serp_tool],
    system_prompt = "You are a helpful assistant that can search the internet"
)

result = serp_agent.invoke(
    {"messages" : [{"role" : "user", "content" : "What is the weather in mannheim?"}]}
)
result["messages"][-1].content

In [ ]:
result

In [ ]:
for chunk in serp_agent.stream(
    {"messages" : [{"role" : "user", "content" : "What is the weather in mannheim?"}]}
):
    agent_result = chunk.get('agent')
    if agent_result:
        agent_msgs = agent_result.get("messages")

        if not agent_msgs:
            continue
        txt_response = agent_msgs[-1].content

        if txt_response:
            print(txt_response)